# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns
from trading_bot.agent import Agent

Using TensorFlow backend.


In [2]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

In [34]:
tlist = ['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'INFRATEL', 'BRITANNIA', 'CIPLA', 'COALINDIA']

In [36]:
data = pd.DataFrame(columns = ['ticker','strategy','ep_count','year','investment','returns','required capital','profit','profit percentage'])

In [39]:
def fn(ticker,strategy,ep_count,year):
    stock = '{}'.format(ticker)
    model_name = '{}_{}_{}'.format(ticker,strategy,ep_count)
    test_stock = 'data/Nifty50/Split/{}_{}.csv'.format(ticker,year)
    window_size = 10
    debug = True

    agent = Agent(window_size, pretrained=True, model_name=model_name)

    # read csv into dataframe
    df = pd.read_csv(test_stock)
    # df = df.iloc[:55]
    # filter out the desired features
    df = df[['Date', 'Adj Close']]
    # rename feature column names
    df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
    # convert dates from object to DateTime type
    dates = df['date']
    dates = pd.to_datetime(dates, infer_datetime_format=True)
    df['date'] = dates

    df.head()



    coloredlogs.install(level='DEBUG')
    switch_k_backend_device()

    test_data = get_stock_data(test_stock)
    initial_offset = test_data[1] - test_data[0]

    test_result, history = evaluate_model(agent, test_data, window_size, debug)
    show_eval_result(model_name, test_result, initial_offset)

    def visualize(df, history, title="trading session"):
        # add history to dataframe
        position = [history[0][0]] + [x[0] for x in history]
        actions = ['HOLD'] + [x[1] for x in history]
        df['position'] = position
        df['action'] = actions

        # specify y-axis scale for stock prices
        scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)

        # plot a line chart for stock positions
        actual = alt.Chart(df).mark_line(
            color='green',
            opacity=0.5
        ).encode(
            x='date:T',
            y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
        ).interactive(
            bind_y=False
        )

        # plot the BUY and SELL actions as points
        points = alt.Chart(df).transform_filter(
            alt.datum.action != 'HOLD'
        ).mark_point(
            filled=True
        ).encode(
            x=alt.X('date:T', axis=alt.Axis(title='Date')),
            y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
            color='action'
        ).interactive(bind_y=False)

        # merge the two charts
        chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)

        return chart

    chart = visualize(df, history, title=test_stock)
#         chart

    cap = [0]
    inv = 0
    ret = 0
    b = 0
    for i in range(len(df)):
        if df.iloc[i]['action']=='BUY':
            cap.append(cap[-1]+df.iloc[i]['actual'])
            inv+=df.iloc[i]['actual']
            b+=1
        if df.iloc[i]['action']=='SELL' and b>0:
            cap.append(cap[-1]-df.iloc[i]['actual'])
            ret += df.iloc[i]['actual']
            b-=1

    req_cap = max(cap)

    prof = ret+(df['action'].value_counts().get('BUY',0)-df['action'].value_counts().get('SELL',0))*df.iloc[-1]['actual']-inv

    return pd.DataFrame([[ticker,strategy,ep_count,year,inv,ret,req_cap, prof, (prof/req_cap)*100]],columns=['ticker','strategy','ep_count','year','investment','returns','required capital','profit','profit percentage'])


In [ ]:
for ticker in tlist:
    for ep_count in range(10,60,10):
        for year in ['2018','2019']:
            data = data.append(fn(ticker,'t-dqn',ep_count,year))

2020-09-28 16:22:24 Tornado root[30599] DEBUG switching to TensorFlow for CPU
2020-09-28 16:22:24 Tornado root[30599] DEBUG Buy at: $393.09
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $390.88
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $400.96
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $403.12 | Position: +$10.03
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $416.85 | Position: +$25.97
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $415.96
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $414.19
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $416.50
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $418.52 | Position: +$17.56
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $416.35 | Position: +$0.39
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $413.70
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $416.60
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $426.29 | Position: +$12.10
2020-09-28 1

2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $355.96 | Position: -$2.36
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $359.75 | Position: -$4.18
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $360.93 | Position: +$3.20
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $364.37 | Position: +$12.00
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $367.37 | Position: +$10.92
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $363.29
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $364.32 | Position: +$5.16
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $354.14 | Position: -$5.36
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $360.68 | Position: +$12.39
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $358.08
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $363.54
2020-09-28 16:22:25 Tornado root[30599] DEBUG Buy at: $363.98
2020-09-28 16:22:25 Tornado root[30599] DEBUG Sell at: $377.45 | Position: +$13.82
2020-09-28

2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $376.93
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $373.77
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $377.77
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $374.41
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $372.98
2020-09-28 16:22:28 Tornado root[30599] DEBUG Sell at: $375.90 | Position: -$8.65
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $372.83
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $370.16
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $380.74
2020-09-28 16:22:28 Tornado root[30599] DEBUG Sell at: $384.84 | Position: +$7.91
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $391.66
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $386.13
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $380.79
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $376.59
2020-09-28 16:22:28 Tornado root[30599] DEBUG Buy at: $369.57
2020-09-28 16:22:28 Tornado ro

2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $410.15 | Position: +$19.03
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $397.84
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $393.24
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $400.36
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $400.36
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $408.17 | Position: +$22.09
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $410.54 | Position: +$23.97
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $404.42 | Position: +$17.99
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $412.97 | Position: +$28.17
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $412.27 | Position: +$25.01
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $407.68
2020-09-28 16:22:29 Tornado root[30599] DEBUG Buy at: $400.86
2020-09-28 16:22:29 Tornado root[30599] DEBUG Sell at: $402.54 | Position: +$14.24
2020-09-28 16:22:29 Tornado root[30599] DEBUG B

2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $416.50
2020-09-28 16:22:33 Tornado root[30599] DEBUG Sell at: $418.52 | Position: +$2.02
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $416.35
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $413.70
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $416.60
2020-09-28 16:22:33 Tornado root[30599] DEBUG Sell at: $426.29 | Position: +$9.93
2020-09-28 16:22:33 Tornado root[30599] DEBUG Sell at: $409.42 | Position: -$4.28
2020-09-28 16:22:33 Tornado root[30599] DEBUG Sell at: $430.86 | Position: +$14.26
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $429.48
2020-09-28 16:22:33 Tornado root[30599] DEBUG Sell at: $421.22 | Position: -$8.26
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $421.71
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $411.09
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $401.50
2020-09-28 16:22:33 Tornado root[30599] DEBUG Buy at: $398.16
2020-09-28 16:22:33 Tornado roo

...

2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2640.54
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2634.62
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2665.13
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2689.63
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2734.52
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2676.58
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2665.42
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2661.19
2020-09-28 16:51:01 Tornado root[30599] DEBUG Sell at: $2676.06 | Position: -$202.49
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2643.86
2020-09-28 16:51:01 Tornado root[30599] DEBUG Sell at: $2714.79 | Position: -$151.43
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2645.79
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at: $2631.69
2020-09-28 16:51:01 Tornado root[30599] DEBUG Sell at: $2627.55 | Position: -$230.06
2020-09-28 16:51:01 Tornado root[30599] DEBUG Buy at

2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $603.07
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $592.51
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $595.52
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $589.41
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $581.90
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $582.35
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $578.80
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $581.90
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $574.06
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $571.64
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $576.43
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $572.58
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $570.41
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $573.02
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $570.31
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $552.05
2020-09-

2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $648.69
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $649.09
2020-09-28 17:01:03 Tornado root[30599] DEBUG Sell at: $631.34 | Position: +$105.29
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $633.47
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $634.01
2020-09-28 17:01:03 Tornado root[30599] DEBUG Sell at: $634.26 | Position: +$120.65
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $611.06
2020-09-28 17:01:03 Tornado root[30599] DEBUG Sell at: $617.70 | Position: +$100.58
2020-09-28 17:01:03 Tornado root[30599] DEBUG Sell at: $642.44 | Position: +$117.04
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $634.41
2020-09-28 17:01:03 Tornado root[30599] DEBUG Sell at: $635.11 | Position: +$108.61
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $628.51
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $624.40
2020-09-28 17:01:03 Tornado root[30599] DEBUG Buy at: $616.12
2020-09-28 17:01:03 To

2020-09-28 17:15:49 Tornado root[30599] DEBUG Sell at: $560.29 | Position: +$44.18
2020-09-28 17:15:49 Tornado root[30599] DEBUG Sell at: $560.88 | Position: +$39.86
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $559.79
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $558.65
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $553.34
2020-09-28 17:15:49 Tornado root[30599] DEBUG Sell at: $550.91 | Position: +$9.02
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $542.14
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $551.36
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $551.06
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $545.61
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $539.61
2020-09-28 17:15:49 Tornado root[30599] DEBUG Sell at: $560.73 | Position: +$20.43
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $548.39
2020-09-28 17:15:49 Tornado root[30599] DEBUG Buy at: $560.04
2020-09-28 17:15:49 Tornado root[30599] DEBUG Sel

2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $477.31 | Position: -$37.45
2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $466.79 | Position: -$50.77
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $462.80
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $463.05
2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $468.78 | Position: -$30.07
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $460.40
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $454.27
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $451.88
2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $459.86 | Position: -$23.39
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $460.06
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $457.06
2020-09-28 17:15:50 Tornado root[30599] DEBUG Buy at: $466.59
2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $466.39 | Position: -$8.13
2020-09-28 17:15:50 Tornado root[30599] DEBUG Sell at: $464.54 | Position: -$12.52
2020-09

In [33]:
data

,ticker,strategy,ep_count,year,investment,returns,required capital,profit,profit percentage
0,ADANIPORTS,t-dqn,50,2018,15.92415,0,15.92415,0.422875,2.655558
